In [2]:
''' 
 Author: Yoonhyuck WOO / JBNU_Industrial Information system Engineering
 Date; 9. 24. 2021 - 11. 13. 2021
 Title: Artificial Intelligence_Project 2
 Professor: Seung-Hoon Na
 Reference: https://cpm0722.github.io/pytorch-implementation/transformer
            https://pytorch.org/tutorials/beginner/transformer_tutorial.html'''

' \n Author: Yoonhyuck WOO / JBNU_Industrial Information system Engineering\n Date; 7. . 2021 - 2. 5. 2021\n Title: Artificial Intelligence_Project 2\n Professor: Seung-Hoon Na'

In [4]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer

# Encoder
 - Generate context (vector made by Encoding)
 - compress all of the information in sentences without leakage

# Decoder
- make a sentence from sentence, context


# Self-Attention
- 'Self': find attention in a same sentence
- Attention: guess what a word(ex. it or the thrid person singular) points in a sentecne
- Query: now token
- Key: target token to find attention
- Value: same Key

In [5]:
class TransformerModel(nn.Module):
    
    def __init__(self, ntoken, d_model, nhead, d_hid, nlayers, dropout = 0.5):
        
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout) # include information of word's location
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)
        
        self.init_weights()
    
    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform(-initrange, initrange)
        
    def forward(self, src, src_mask):
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: 
        """
        